In [1]:
import sys
import os 
sys.path.append('..')  #adds the Root Directory to the system path
from BL_CalciumAnalysis.image_analysis_methods import ImageAnalysis

In [2]:
test_path = '/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis'
try:
    files = os.listdir(test_path)
    print("Files in directory:", files)
except Exception as e:
    print("Error accessing directory:", e)

Files in directory: ['g21_12072023_estim_10hz_na_blk', 'g22_12072023_estim_10hz_na_blk', 'g23_12072023_estim_10hz_na_blk']


In [6]:
project_folder = '/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis'
analysis = ImageAnalysis(project_folder)

# Since the folders are directly inside the project_folder, you don't need to append any subdirectory name
data_files = analysis.list_files('') 
print(data_files) #print the list of files in the project folder 

directories = analysis.list_directories()
print(directories) #print the list of directories in the project folder 

['/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis/g21_12072023_estim_10hz_na_blk/g21_12072023_estim_10hz_na_blk_fluor_combined.tif', '/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis/g21_12072023_estim_10hz_na_blk/g21_12072023_estim_10hz_na_blk_fluor.csv', '/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis/g21_12072023_estim_10hz_na_blk/._g21_12072023_estim_10hz_na_blk_fluor.csv', '/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis/g22_12072023_estim_10hz_na_blk/g22_12072023_estim_10hz_na_blk_fluor_combined.tif', '/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis/g22_12072023_estim_10hz_na_blk/g22_12072023_estim_10hz_na_blk_fluor.csv', '/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis/g23_12072023_estim_10hz_na_blk/g23_12072023_estim_10hz_na_blk.tif', '/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis/g23_12072023_estim_10hz_na_blk/g23_12072023_estim_10hz_na_blk_fluor.csv']
['g21_12072023_estim_10hz_na_blk', 'g22_12072023_estim_10hz_na_blk', 'g23_1